In [152]:
#@title # Create a Character Card v3 incl. World (SillyTavern) { display-mode: "form" }
#@markdown ⬅️❗Run this code first to initialize all parameters.

#@markdown Throughout this notebook, you will create your CharCard_v3 step by step:
#@markdown 1. Step: Creator Metadata
#@markdown 2. Step: System Prompt & Jailbreak (Optional)
#@markdown 3. Step: Character Personality (Essential)\
#@markdown    3.1. Step: Character Personality (Essential)
#@markdown 4. Step: Personality Depth (Optional)
#@markdown 5. Step: World & Complexity (Optional)\
#@markdown    5.1. Step: Keywords and Description

#@markdown How many tokens your character generates (gpt-3.5-turbo)

#@markdown 6. Step: Finalization\
#@markdown    6.1. Step: Save and Download


import os
import sys

sys.stdout = open(os.devnull, 'w')

!pip show tiktoken || pip install tiktoken
!pip show panel || pip install panel
!pip show bokeh || pip install jupyter_bokeh
!pip show timezonefinder || pip install timezonefinder

import pytz
import json
import time
import param
import tiktoken
import geocoder
import threading
import panel as pn
from pathlib import Path
from datetime import datetime
from functools import partial
from zoneinfo import ZoneInfo

eu = ZoneInfo("Europe/Berlin")

tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

pn.extension(
    theme='dark',
    raw_css=["textarea { resize: none; }"]
)

""" Serves for the modular definition and organization of properties. """
class CreatorMeta(param.Parameterized):
    creator = param.String(
        default="", doc="Character Card creator"
        )
    creator_notes = param.String(
        default="", doc="Notes from the creator"
        )
    character_version = param.String(
        default="1.0.0", doc="Version number of the Character Card"
        )
    tags = param.String(
        default="", doc="Example: Elf, Fantasy, Medieval"
        )

class PromptOverrides(param.Parameterized):
    system_prompt = param.String(
        default="", doc="Custom system prompt that replaces default instructions."
        )
    post_history_instructions = param.String(
        default="", doc="“Jailbreak” can allow filters to be bypassed."
        )

class Personality(param.Parameterized):
    char_name = param.String(
        default="", doc="Character name"
        )
    description = param.String(
        default="", doc="Character description"
        )
    personality = param.String(
        default="", doc="Character personality traits\nExample: shy + intelligent + friendly"
        )
    scenario = param.String(
        default="", doc="Character's scenario or backstory\nDo NOT use <START> here"
        )
    first_mes = param.String(
        default="", doc="Character's first message or greeting\nDo NOT use <START> here"
        )
    mes_example = param.String(
        default="", doc="Example message from the character\nStart each example message with <START>"
        )

class CharExtens_0(param.Parameterized):
    depth_prompt = param.String(
        default="", doc="Prompt to deepen the character's personality"
        )

class CharExtens_1(param.Parameterized):
    depth = param.Integer(4, bounds=(1, 6))
    talkativeness = param.Number(0.5, bounds=(0, None), softbounds=(0, 1))

class CharAlt(param.Parameterized):
    alternate_greetings = param.String(
        default="", doc="Alternative greetings from the character"
        )

class World(param.Parameterized):
    world = param.String(default=None, doc="Name of the world. Example: Eldoria")

class WorldKeys(param.Parameterized):
    keys = param.String(
        default="", doc="Keywords or categories for the world description\nExample: eldoria, forest, magical forest"
        )
    content = param.String(
        default="", doc="<START>\n{{user}}: What is Eldoria?\n{{char}}: Eldoria is an ancient, magical forest filled with mysterious creatures and glowing plants. It is considered the heart of nature magic and holds many hidden paths and mystical places only the bravest explore."
        )


def create_widgets(param_instance, max_items_per_row=2, exclude_counters=None, y=100, x=400, only_param=None):
    """ Dynamically creates widgets for input and display of parameter values."""
    if exclude_counters is None:
        exclude_counters = []

    widgets = []
    row = []

    for param_name, param_obj in param_instance.param.objects("existing").items():

        if only_param is not None and param_name not in only_param:
            continue

        if isinstance(param_obj, param.String) and param_name not in ['name']:
            label = pn.pane.Markdown(f"### {param_name.replace('_', ' ').capitalize()}")
            placeholder_text = param_obj.doc if param_obj.doc else f"Enter your {param_name} here..."
            text_area = pn.widgets.TextAreaInput(
                value=getattr(param_instance, param_name),
                placeholder=placeholder_text,
                height=y,
                width=x,
                max_length=10000
            )

            if param_name not in exclude_counters:
                char_counter = pn.pane.Markdown("**0 characters ~ 0 Tokens by Tiktoken**")

                def update_char_count(event, char_counter, param_name=param_name):
                    text = event.new
                    setattr(param_instance, param_name, text)
                    char_count = len(text)
                    token_count = len(tokenizer.encode(text))
                    char_counter.object = f"**{char_count} characters ~ {token_count} Tokens by Tiktoken**"

                text_area.param.watch(partial(update_char_count, char_counter=char_counter), 'value')

                row.append(pn.Column(label, text_area, char_counter))
            else:
                def update_value_no_counter(event, pn_name=param_name):
                    setattr(param_instance, pn_name, event.new)

                text_area.param.watch(partial(update_value_no_counter, pn_name=param_name), 'value')

                row.append(pn.Column(label, text_area))

            if len(row) == max_items_per_row:
                widgets.append(pn.Row(*row))
                row = []

    if row:
        widgets.append(pn.Row(*row))

    return widgets


def read_param_values(param_instance, only_params=None):
    """
    Reads all parameter values from the param_instance and returns them as a dictionary.
    If only_params is specified (list of strings), only those parameters will be read.
    """
    values_dict = {}

    for param_name, param_obj in param_instance.param.objects("existing").items():
        if isinstance(param_obj, param.String) and param_name not in ['name']:
            if only_params is not None and param_name not in only_params:
                continue

        values_dict[param_name] = getattr(param_instance, param_name)

    return values_dict

"""
Creates a Panel layout with a confirmation button and an output display area.
"""
def create_confirm_button():

    button = pn.widgets.Button(name='Confirm Input')
    output = pn.pane.Markdown("")

    def animate_output():
        confirm_time = datetime.now(eu).strftime("%H:%M:%S")
        for dots in [".", "..", "..."]:
            output.object = dots
            time.sleep(0.2)
            pn.io.push_notebook()  # Notebook manuell aktualisieren

        output.object = f"### <span style='color:green;'>**Input confirmed [{confirm_time}]**</span>"

    def on_button(event):
        pn.state.execute(animate_output)  # Sicherstellen, dass Animation nicht blockiert

    button.on_click(on_button)
    return pn.Column(button, output)

"""
Dynamically creates a dictionary of button panels, where
each key is an index and each value is a confirmation button widget.
"""
button_panels = {i: create_confirm_button() for i in range(1, 10)}


# Instance of the classes
creator_meta = CreatorMeta()
jailbreak = PromptOverrides()
personality = Personality()
char_extens = CharExtens_0()
char_extens_1 = CharExtens_1()
char_alt = CharAlt()
world_name = World()


# Widgets for all parameters
widgets_0 = create_widgets(creator_meta, y=35, exclude_counters=['creator', 'creator_notes', 'character_version', 'tags'])
widgets_1 = create_widgets(jailbreak, x=500)
widgets_2 = create_widgets(personality, y=35, only_param=['char_name'])
widgets_3 = create_widgets(personality, x=600, y=300, only_param=['description', 'personality'])
widgets_3_1 = create_widgets(personality, x=600, y=300, only_param=['scenario', 'first_mes', 'mes_example'])
widgets_4 = create_widgets(char_extens, x=600, y=300, only_param=['depth_prompt'])
widgets_5 = create_widgets(char_alt, x=600, y=300)
widgets_6 = create_widgets(world_name, y=35, exclude_counters=['world'])

sys.stdout = sys.__stdout__
pn.pane.Markdown("The parameters are now initialized. You can now proceed to step 1.")


Markdown(str)

In [153]:
#@title 1. Step: Creator Metadata { display-mode: "form" }
#@markdown In this section, you can specify information about your CharCard.\
#@markdown Entries in this area do not generate tokens.
pn.Column(
    pn.pane.Markdown("## Meta Daten deiner CharCard"),
    *widgets_0,
    button_panels[1]
)


Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=35, max_length=10000, placeholder='Character Card creator', sizing_mode='fixed', width=400)
        [1] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=35, max_length=10000, placeholder='Notes from the creator', sizing_mode='fixed', width=400)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=35, max_length=10000, placeholder='Version number o..., sizing_mode='fixed', value='1.0.0', width=400)
        [1] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=35, max_length=10000, placeholder='Example: Elf, ..., sizing_mode='fixed', width=400)
    [3] Column
        [0] Button(name='Confirm Input')
        [1] Markdown(str)

In [137]:
#@title 2. Step: System Prompt & Jailbreak (Optional) { display-mode: "form" }
#@markdown In this section, you can optionally enter a "Jailbreak" (not recommended).


pn.Column(
    pn.pane.Markdown("## CharCard"),
    *widgets_1,
    button_panels[2]
)

Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=100, max_length=10000, placeholder='Custom system p..., sizing_mode='fixed', width=500)
            [2] Markdown(str)
        [1] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=100, max_length=10000, placeholder='“Jailbreak” c..., sizing_mode='fixed', width=500)
            [2] Markdown(str)
    [2] Column
        [0] Button(name='Confirm Input')
        [1] Markdown(str)

In [138]:
#@title 3. Step: Character Personality (Essential!) { display-mode: "form" }
#@markdown In this step, you define your character's personality profile.

pn.Column(
    pn.pane.Markdown(
        "# Persönlichkeit des Charakters\n"
        ),
    *widgets_2,
    *widgets_3,
    button_panels[3],
    height_policy="fit"
    )

Column(height_policy='fit')
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=35, max_length=10000, placeholder='Character name', sizing_mode='fixed', width=400)
            [2] Markdown(str)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=300, max_length=10000, placeholder='Character description', sizing_mode='fixed', width=600)
            [2] Markdown(str)
        [1] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=300, max_length=10000, placeholder='Character personality t..., sizing_mode='fixed', width=600)
            [2] Markdown(str)
    [3] Column
        [0] Button(name='Confirm Input')
        [1] Markdown(str)

In [144]:
#@title 3.1. Step: Character Personality Context (Essential!) { display-mode: "form" }
#@markdown When describing actions, use "*non-verbal actions*".\
#@markdown Some of these fields require a specific format.\
#@markdown Formatting guidelines:
#@markdown + Use `{{char}}` instead of the character name.
#@markdown + Use `{{user}}` instead of the username.

#@markdown Example:
#@markdown ```
#@markdown <START>
#@markdown {{user}}: Hey Aqua, how’s your adventure going?
#@markdown {{char}}: *laughs loudly* Oh, my adventures? They mostly end in chaos, but hey, that’s part of the fun, right?
#@markdown ```

pn.Column(
    pn.pane.Markdown(
        "# Charakter Kontext\n"
        ),
    *widgets_3_1,
    button_panels[4],
    )


Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=300, max_length=10000, placeholder="Character's s..., sizing_mode='fixed', width=600)
            [2] Markdown(str)
        [1] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=300, max_length=10000, placeholder="Character's f..., sizing_mode='fixed', width=600)
            [2] Markdown(str)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=300, max_length=10000, placeholder='Example message f..., sizing_mode='fixed', width=600)
            [2] Markdown(str)
    [3] Column
        [0] Button(clicks=2, name='Confirm Input')
        [1] Markdown(str)

In [143]:
#@title 4. Step: Personality Depth (Optional) { display-mode: "form" }
#@markdown Adding depth to your character's personality makes them maintain their personality more consistently.\
#@markdown * Less freedom in character development.\
#@markdown * You can add an alternative greeting message.
pn.Column(
    pn.pane.Markdown("## Ersteller\n"),
    pn.Row(
        *widgets_4,
        *widgets_5
    ),
    pn.Param(char_extens_1.param, show_name=False),
    button_panels[5]
)

Column
    [0] Markdown(str)
    [1] Row
        [0] Row
            [0] Column
                [0] Markdown(str)
                [1] TextAreaInput(height=300, max_length=10000, placeholder="Prompt to deepen t..., sizing_mode='fixed', width=600)
                [2] Markdown(str)
        [1] Row
            [0] Column
                [0] Markdown(str)
                [1] TextAreaInput(height=300, max_length=10000, placeholder='Alternative g..., sizing_mode='fixed', width=600)
                [2] Markdown(str)
    [2] Param(CharExtens_1, name='CharExtens_', show_name=False)
    [3] Column
        [0] Button(name='Confirm Input')
        [1] Markdown(str)

In [145]:
#@title 5. Step: World & Complexity (Optional) { display-mode: "form" }
#@markdown Here, you specify the name of the world in which your character lives.\
#@markdown * Decide how complex your world should be.\
#@markdown This is only recommended for models that can handle a large context.\
#@markdown (My recommendation: Claude 3.5 Haiku)
class ValueInfo(param.Parameterized):
    value_infos = param.Integer(4, bounds=(1, 12))

value_info = ValueInfo()

pn.Column(
    pn.pane.Markdown(
        "## Number of World Details\n"
        "Warning! You will need a lot of information here.\n"
        "If you want your world to be complex, I recommend using 4 world details."
        ),
    pn.Param(value_info.param, show_name=False),
    *widgets_6,
    button_panels[6]
    )

Column
    [0] Markdown(str)
    [1] Param(ValueInfo, show_name=False)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] TextAreaInput(height=35, max_length=10000, placeholder='Name of the w..., sizing_mode='fixed', width=400)
    [3] Column
        [0] Button(name='Confirm Input')
        [1] Markdown(str)

In [147]:
#@title 5.1. Step: Keywords and Description { display-mode: "form" }
#@markdown Warning: *Only execute this step if you have completed Step 5!*

value = value_info.value_infos
name = world_name.world
world_keys_list = []

if name is not None and not len(name) == 0:
    def world_value(value, name):

        world_keys = WorldKeys()
        world_keys_list.append(world_keys)

        widgets_7 = create_widgets(world_keys, y = 60, only_param=['keys'])
        widgets_8 = create_widgets(world_keys, x=600, y=300, only_param=['content'])

        return pn.Column(
            pn.pane.Markdown(f"## {value+1}. Description of {name}"),
            pn.Column(
                *widgets_7,
                *widgets_8
            )
        )

    panels = []
    for i in range(value):
        panels.append(world_value(i, name))


else:
    panels = pn.pane.Markdown('You did not provide a world name!')

pn.Column(*panels, button_panels[7])


Column
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Column
                    [0] Markdown(str)
                    [1] TextAreaInput(height=60, max_length=10000, placeholder='Keywords or categories fo..., sizing_mode='fixed', width=400)
                    [2] Markdown(str)
            [1] Row
                [0] Column
                    [0] Markdown(str)
                    [1] TextAreaInput(height=300, max_length=10000, placeholder='<START>\n{{user}}: What i..., sizing_mode='fixed', width=600)
                    [2] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Row
                [0] Column
                    [0] Markdown(str)
                    [1] TextAreaInput(height=60, max_length=10000, placeholder='Keywords or categories fo..., sizing_mode='fixed', width=400)
                    [2] Markdown(str)
            [1] Row
                [0] Column
                    [0] Markdown(str)
                    [1] TextAreaInput(height=300, max_length=10000, placeholder='<START>\n{{user}}: What i..., sizing_mode='fixed', width=600)
                    [2] Markdown(str)
    [2] Column
        [0] Button(clicks=2, name='Confirm Input')
        [1] Markdown(str)

In [148]:
#@title #####Debug (You can ignore this section) { display-mode: "form" }
button = pn.widgets.Button(name='Read Parameters')

def on_button_click(event):
    button.disabled = True
    all = [
        read_param_values(creator_meta),
        read_param_values(jailbreak),
        read_param_values(personality),
        read_param_values(char_extens),
        read_param_values(char_extens_1),
        read_param_values(char_alt),
        read_param_values(world_name),
    ]
    if 'world_keys_list' in globals():
        for world_keys in world_keys_list:
            all.append(read_param_values(world_keys))

    print(all)

button.on_click(on_button_click)

pn.Column(button)

Column
    [0] Button(name='Read Parameters')

In [149]:
#@title Learn how many tokens your character generates (gpt-3.5-turbo) { display-mode: "form" }
#@markdown 2048 tokens are recommended, but some models (like Claude) can handle an input of up to 100k tokens.\
#@markdown - From experience: Less is more.\
#@markdown (Remember that memory also requires tokens.)\
#@markdown Rough example: Total 8192 tokens - Personality 2048 tokens = Memory 6144 tokens.\
#@markdown The total number of tokens depends on your model and settings.

# Function to calculate the total number of tokens for a parameter
def total_token_count(param_instance, encoding):
    total = 0
    for param_name, param_obj in param_instance.param.objects("existing").items():
        if isinstance(param_obj, param.String) and param_name not in ['name']:
            wert = getattr(param_instance, param_name)
            if wert:
                tokens = len(encoding.encode(wert))
                total += tokens
    return total

encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')

gesamt_tokens = (
    total_token_count(personality, encoding) +
    total_token_count(jailbreak, encoding) +
    total_token_count(char_extens, encoding) +
    total_token_count(char_extens_1, encoding) +
    total_token_count(char_alt, encoding)
)
if 'world_keys_list' in globals():
    for world_keys in world_keys_list:
      gesamt_tokens += (total_token_count(world_keys, encoding))



if gesamt_tokens >= 2048:
    color = "yellow"
else:
    color = "green"



markdown_text = f"# Gesamt: <span style='color:{color};'>Tokens {gesamt_tokens}</span>"

pn.pane.Markdown(markdown_text).servable()

Markdown(str)

In [150]:
#@title 6. Step: Finalization { display-mode: "form" }
#@markdown Your entries will now be converted into a `.json` format.\
#@markdown Start the code and save it in Step 6.1.

all_para = [
    read_param_values(creator_meta),
    read_param_values(jailbreak),
    read_param_values(personality),
    read_param_values(char_extens),
    read_param_values(char_extens_1),
    read_param_values(char_alt),
    read_param_values(world_name)
    ]

world_para = []
if 'world_keys_list' in globals():
    for world_keys in world_keys_list:
        world_para.append(read_param_values(world_keys))

day = datetime.now(eu).strftime("%Y-%m-%d")
time = datetime.now(eu).strftime("%Hh %Mm %Ss %Z")
# Initialisieren der JSON-Struktur
json_data = {
    "name": "",
    "description": "",
    "personality": "",
    "first_mes": "",
    "avatar": "none",
    "chat": "",
    "mes_example": "",
    "scenario": "",
    "create_date": f"{day} @{time}",
    "talkativeness": "",
    "fav": False,
    "creatorcomment": "",
    "spec": "chara_card_v3",
    "spec_version": "3.0",
    "data": {
        "name": "",
        "description": "",
        "personality": "",
        "scenario": "",
        "first_mes": "",
        "mes_example": "",
        "creator_notes": "",
        "system_prompt": "",
        "post_history_instructions": "",
        "tags": [],
        "creator": "",
        "character_version": "",
        "alternate_greetings": [],
        "extensions": {
            "talkativeness": "0.5",
            "fav": False,
            "world": "",
            "depth_prompt": {
                "prompt": "",
                "depth": 4,
                "role": "system"
            }
        },
        "character_book": {
            "entries": [],
            "name": ""
        },
        "group_only_greetings": []
    },
    "tags": []
}


def assign_parameters(params, json_struct):
    for param in params:

        name = param.get('name')

        if name.startswith('CreatorMeta'):

            json_struct['creatorcomment'] = param.get('creator_notes', '')
            json_struct['data']['creator'] = param.get('creator', '')
            json_struct['data']['creator_notes'] = param.get('creator_notes', '')
            json_struct['data']['character_version'] = param.get('character_version', '')

            tags = param.get('tags', '')
            if isinstance(tags, list):
                json_struct['data']['tags'] = tags
                json_struct['tags'] = tags
            elif isinstance(tags, str):
                json_struct['data']['tags'] = [tags]
                json_struct['tags'] = tags

        elif name.startswith('PromptOverrides'):
            json_struct['data']['system_prompt'] = param.get('system_prompt', '')
            json_struct['data']['post_history_instructions'] = param.get('post_history_instructions', '')

        elif name.startswith('Personality'):
            json_struct['name'] = param.get('char_name', '')
            json_struct['description'] = param.get('description', '')
            json_struct['personality'] = param.get('personality', '')
            json_struct['scenario'] = param.get('scenario', '')
            json_struct['first_mes'] = param.get('first_mes', '')
            json_struct['mes_example'] = param.get('mes_example', '')

            json_struct['data']['name'] = param.get('char_name', '')
            json_struct['data']['description'] = param.get('description', '')
            json_struct['data']['personality'] = param.get('personality', '')
            json_struct['data']['scenario'] = param.get('scenario', '')
            json_struct['data']['first_mes'] = param.get('first_mes', '')
            json_struct['data']['mes_example'] = param.get('mes_example', '')

        elif name.startswith('CharExtens'):
            for key, value in param.items():
                if key == 'name':
                    continue
                if key == 'depth_prompt' and isinstance(value, str):
                    json_struct['data']['extensions']['depth_prompt']['prompt'] = value
                elif key == 'depth':
                    json_struct['data']['extensions']['depth'] = value
                elif key == 'talkativeness':
                    json_struct['talkativeness'] = f"{value}"
                    json_struct['data']['extensions']['talkativeness'] = f"{value}"

        elif name.startswith('World'):
            for key, value in param.items():
                if key == 'name':
                    continue
                elif key == 'world':
                    json_struct['data']['extensions']['world'] = value
                    json_struct['data']['character_book']['name'] = value

        elif name.startswith('CharAlt'):
            alternate_greetings = param.get('alternate_greetings', [])
            if isinstance(alternate_greetings, str):
                json_struct['data']['alternate_greetings'].append(alternate_greetings)
            elif isinstance(alternate_greetings, list):
                json_struct['data']['alternate_greetings'].extend(alternate_greetings)

def add_world_entries(world_keys, json_struct):
    id = 0

    for world_key in world_para:
        key = world_key.get('keys')
        content = world_key.get('content')

        entry = {
            "id": id,
            "keys": [key],
            "secondary_keys": [],
            "comment": "",
            "content": content,
            "constant": False,
            "selective": True,
            "insertion_order": 100,
            "enabled": True,
            "position": "before_char",
            "use_regex": True,
            "extensions": {
                "position": 0,
                "exclude_recursion": False,
                "display_index": 0,
                "probability": 0,
                "useProbability": True,
                "depth": 4,
                "selectiveLogic": 0,
                "group": "",
                "group_override": False,
                "group_weight": 100,
                "prevent_recursion": False,
                "delay_until_recursion": False,
                "scan_depth": None,
                "match_whole_words": None,
                "use_group_scoring": False,
                "case_sensitive": None,
                "automation_id": "",
                "role": 0,
                "vectorized": False,
                "sticky": 0,
                "cooldown": 0,
                "delay": 0
            }
        }
        json_struct['data']['character_book']['entries'].append(entry)

        id+=1

assign_parameters(all_para, json_data)

if 'world_keys_list' in globals():
    add_world_entries(world_para, json_data)

pn.pane.Markdown("You can now save and download your CharCard.")


Markdown(str)

In [ ]:
#@title #####Debug (You can ignore this section) { display-mode: "form" }
pn.pane.Markdown(json.dumps(json_data, indent=4, ensure_ascii=False))

In [151]:
#@title 6.1. Step: Save and Download { display-mode: "form" }
#@markdown Your CharCard will be saved in the CharCard folder and downloaded.

from pathlib import Path
from google.colab import files

base_name = personality.char_name if personality.char_name else "CharCard"
base_filename = f"{base_name}.json"
directory = Path('CharCard')
filename = directory / base_filename

directory.mkdir(parents=True, exist_ok=True)

def get_unique_filename(filepath):
    counter = 1
    new_filepath = filepath
    while new_filepath.exists():
        new_filename = f"{filepath.stem}_{counter}{filepath.suffix}"
        new_filepath = filepath.parent / new_filename
        counter += 1
    return new_filepath

unique_filename = get_unique_filename(filename)

with unique_filename.open('w', encoding='utf-8') as file:
    json.dump(json_data, file, indent=4, ensure_ascii=False)

files.download(str(unique_filename))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>